## Importing Librarys and Setting up the Environment 



In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os 
import pandas as pd 
import numpy as np
import copy
import re
import string
path = 'gdrive/MyDrive/2020_21_2/Temalabor_1/processed_popdata/'

## Function definitions
1. Function defintion of trim_all_columns(df) 


In [ ]:
# Function1.: Trimming  all leading and trailing whitespaces, apply str to the whole dfs

def trim_all_columns(df):

    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    
    df = df.applymap(trim_strings)
    df = df.applymap(str) #converting the whole df datatype into string
    return df


## Premliminary steps 

1.   Loading CSVS
2.   Trimming all leading and trailing whitespaces using 

```
trim_all_columns(df) 
```






In [ ]:
# 1. Loading CSVS

full_pop = pd.read_csv(path + 'populationData_HU_1985-2020_full.csv', dtype=str)
basic_pop = pd.read_csv(path + 'populationData_HU_1985-2020_basic.csv', dtype=str)
election=pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/election.csv', dtype=str)


In [ ]:
# 2. Trimming all leading and trailing whitespaces using trim_all_columns(df) aka Using Function1.:

full_pop = trim_all_columns(full_pop)
basic_pop = trim_all_columns(basic_pop)
election = trim_all_columns(election)

## EDA 


### Setting up for election

In [ ]:
elecYear_pop = full_pop[ (full_pop['year'] == '1990') | (full_pop['year'] == '1994') 
                       | (full_pop['year'] == '1998') | (full_pop['year'] == '2002') 
                       | (full_pop['year'] == '2006') | (full_pop['year'] == '2010') 
                       | (full_pop['year'] == '2014') | (full_pop['year'] == '2018') ]

elecYear_pop.reset_index(drop = True, inplace = True)

In [ ]:
elec_years = sorted(elecYear_pop.year.unique())
print('year' + ' ----- ' + 'num of entries' + ' ----- ' + 'unique num of geonameid' + 'unique num of geoid' )
for year in elec_years: 
  print( year, '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]),  
        '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]['geonameid'].unique()), 
        '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]['geoid'].unique()) ) 

year ----- num of entries ----- unique num of geonameidunique num of geoid
1990 ----- 3091 ----- 3091 ----- 3091
1994 ----- 3149 ----- 3149 ----- 3149
1998 ----- 3153 ----- 3153 ----- 3153
2002 ----- 3167 ----- 3167 ----- 3167
2006 ----- 3174 ----- 3174 ----- 3174
2010 ----- 3176 ----- 3176 ----- 3176
2014 ----- 3177 ----- 3177 ----- 3177
2018 ----- 3177 ----- 3177 ----- 3177


In [ ]:
elecYear_pop

,year,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,1990,Aba,4103,1650,9692,1512,47.0343556977497,18.524754962635
1,1994,Aba,4202,1650,9692,1512,47.0343556977497,18.524754962635
2,1998,Aba,4332,1650,9692,1512,47.0343556977497,18.524754962635
3,2002,Aba,4526,1650,9692,1512,47.0343556977497,18.524754962635
4,2006,Aba,4654,1650,9692,1512,47.0343556977497,18.524754962635
...,...,...,...,...,...,...,...,...
25259,2002,Ősi,2244,1650,9759,2160,47.1539446896795,18.190410278261
25260,2006,Ősi,2190,1650,9759,2160,47.1539446896795,18.190410278261
25261,2010,Ősi,2169,1650,9759,2160,47.1539446896795,18.190410278261
25262,2014,Ősi,2089,1650,9759,2160,47.1539446896795,18.190410278261


## **Examining election dfs**



1.   Creating a variable named *year* from stripping *eventdate* for the purpose of merging with **population** on *year* (and *settlement*)
2.   List item



In [ ]:
#1. Creating a variable named year from stripping eventdate 

election=pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/election.csv', dtype=str)
election = trim_all_columns(election)

year = election['eventdate'].str[0:4] 
election['year'] = year 
cols = election.columns.tolist() # listing all the columns, year is the last column at this moment 
cols = cols[-1:] + cols[:-1]  #putting year as a first col 
election = election[cols] #save this into the election df 

In [ ]:
election

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047408,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0
1047409,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0
1047410,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0
1047411,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0


In [ ]:
election.eventdate.unique()  #There are two events in 1998. Need to drop instances in 1998-05-10 which are also in 1998-05-24 

array(['1990-03-25', '1994-05-08', '1998-05-10', '1998-05-24',
       '2002-04-16', '2006-04-09', '2010-04-11', '2014-04-06',
       '2018-04-08'], dtype=object)

In [ ]:
elec98_1stR = election[election['eventdate'] == '1998-05-10'] # Filter only those when eventdate == '1998-05-10'
elec98_2ndR = election[election['eventdate'] == '1998-05-24'] # Filter only those when eventdate == '1998-05-24'. In 98, there were some wards in settlements where 2nd round election had to be held. We are only interested in second round of those. 

wardnameid_lst_elec98_2ndR = list(elec98_2ndR.wardnameid.unique()) # listing the unique wardnameid we come across in the second round of 98's election
wardnameid_dict_elec98_2ndR = {'wardnameid' : wardnameid_lst_elec98_2ndR} #putting them into a dict with 'wardenameid' key. It's essenital when we wanna find those wardnameid from the 2nd round of 98's election in the first round. (We wanna get rid of those in round 1 of 98's election)

In [ ]:
len(wardnameid_lst_elec98_2ndR) #there are 1077 unique wardnameid in the 2nd round of 98s election 

1077

In [ ]:
row_mask = elec98_1stR.isin(wardnameid_dict_elec98_2ndR).any(1)  #PD Series of booelean value where True pertains to values rows where the value of wardnameid_dict_elec98_2ndR can also be found in wardnameid_dict_elec98_2ndR
print( len(elec98_1stR[row_mask].wardnameid.unique()) )  #checking whether we have 1077 unique wardnameid in elec98_1stR[row_mask]. Yes, indeed. 

print( sorted(elec98_1stR[row_mask].wardnameid.unique()) == sorted(elec98_2ndR.wardnameid.unique()) ) #Checking whether the content of elec98_1stR[row_mask].wardnameid.unique() and elec98_2ndR.wardnameid.unique() is the same => yes, indeed.

elec98_1stR_indices_to_delete = elec98_1stR[row_mask].index # we wanna get the indices of elec98_1stR[row_mask] in order to axe them out in the original election df. 
print( len(elec98_1stR_indices_to_delete) ) #There are 12563 rows which contains in both the wardnameid from elec98_1stR and wardnameid from elec98_2ndR

1077
True
12563


In [ ]:
election = election.drop(elec98_1stR_indices_to_delete) #Dropping those indices from election df and save it again in election  
election.reset_index(drop = True, inplace = True) # reset the indexing of the new election 
election

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034845,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0
1034846,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0
1034847,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0
1034848,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0


In [ ]:
electionPopulation =  pd.merge(election, elecYear_pop, left_on=['settlementid', 'year'], right_on=['geoid', 'year'])
votePop_elig = electionPopulation['value'].astype(np.int64) * 0.8
votePop_elig = votePop_elig.round().astype(np.int64)
electionPopulation['votePop_elig'] = votePop_elig
electionPopulation = electionPopulation.applymap(str)
electionPopulation

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag,votePop_elig
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034845,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034846,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034847,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034848,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56


In [ ]:
len(electionPopulation.name.unique())

3188

## Kitero - partok, partlistak megtalalasa,

partok: MSZP, SZDSZ, MDF, FKGP, Jobbik, LMP, DK, Momentum 
partlistak: jobboldali tomorules: Fidesz-FKGP, Fidesz-MDF, Fidesz KDNP ...


In [ ]:
elsov = election[election.year == '2006']
elsov_partok = elsov[['partylist', 'partyids', 'partylistid']].drop_duplicates(subset=['partylistid'])
elsov_partok

,partylist,partyids,partylistid
456879,Centrum Összefogás Magyarországért,5,27
456880,Magyar Demokrata Fórum,269,47
456881,MCF Roma Összefogás Párt,36,57
456882,Magyar Szocialista Párt,53,59
456883,Jobbik | MIÉP,24 | 43,78
456884,"Szabad Demokraták Szövetsége, a Magyar Liberál...",240,80
456885,"Független Kisgazda-, Nemzeti Egység Párt",125,81
456886,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,86
461223,Magyar Vidék és Polgári Párt,202,11
461230,Magyar Kommunista Munkáspárt,199,89


Fidesz -- 16

KDNP -- 26

SZDSZ -- 240 

MDF --269

MSZP -- 53

FKGP -- 18

Jobbik -- 24

LMP -- 30

DK -- 6

Momentum -- 60760

MSZMP -- 182

MSZDP -- 52 

Agrarszovetseg (ASZ) -- 97

Hazafias Koalicio -- 133

Vallalkozok Partja -- 258

Munkaspart -- 199 

Koztarsasagi Part (KP) -- 153

MIEP -- 43 

MDNP -- 192

Centrum -- 100 (2002-ben) 5 (2006-ban)

PM -- 69 

Egyutt -- 8 (2014-ben) 9 (2018-ban) 

MKKP - 60740



### Fidesz -- 16

In [ ]:
# Searching for '16' string in partyids => Pay attention to '166' which is the partyid of Magyar Fuggetlensegi Part 
lst = list()
for i in election.partyids.unique(): 
  if '16' in i:
    lst.append(i)

print(lst)

['16', '166', '269 | 16', '16 | 269', '16 | 26', '26 | 16']


In [ ]:
#Trial: If we had used df.str.match(...), it woud only show us 5 instances instead of 8 in the following case
settlementid996 = election[ (election.settlementid == '996') & ( (election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) | (election.partyids.str.contains('\| 16 \|')) ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
20835,1990,1,1990-03-25,"1990, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,3,7299,54,Fiatal Demokraták Szövetsége,16,26
148260,1994,4,1994-05-08,"1994, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,71178,41,FIDESZ,16,30
282930,1998,7,1998-05-10,"1998, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,8576,25,Fidesz Magyar Polgári Párt,16,30
398760,2002,12,2002-04-16,"2002, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9225,85,MDF | FIDESZ,269 | 16,110
485433,2006,21,2006-04-09,"2006, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9847,86,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,170
560795,2010,24,2010-04-11,"2010, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,67756,83,KDNP | FIDESZ,26 | 16,172
655851,2014,27,2014-04-06,"2014, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,77325,83,FIDESZ | KDNP,16 | 26,188
855990,2018,48,2018-04-08,"2018, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,69185,86,Kereszténydemokrata Néppárt | Fidesz Magyar Po...,26 | 16,292


### KDNP --26

In [ ]:
# Searching for '26' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '26' in i:
    lst.append(i)

print(lst)

['26', '269', '263', '266', '269 | 16', '16 | 269', '16 | 26', '26 | 16']


In [ ]:
settlementid996 = election[ (election.settlementid == '996') & ( (election.partyids.str.contains('^26$')) | (election.partyids.str.contains('^26 \|')) | (election.partyids.str.contains('\| 26$')) ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
20827,1990,1,1990-03-25,"1990, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,3,7299,6,Kereszténydemokrata Néppárt,26,29
148255,1994,4,1994-05-08,"1994, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,71178,6,Kereszténydemokrata Néppárt,26,29
282928,1998,7,1998-05-10,"1998, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,8576,6,Kereszténydemokrata Néppárt,26,5
485433,2006,21,2006-04-09,"2006, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9847,86,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,170
560795,2010,24,2010-04-11,"2010, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,67756,83,KDNP | FIDESZ,26 | 16,172
655851,2014,27,2014-04-06,"2014, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,77325,83,FIDESZ | KDNP,16 | 26,188
855990,2018,48,2018-04-08,"2018, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,69185,86,Kereszténydemokrata Néppárt | Fidesz Magyar Po...,26 | 16,292


### SZDSZ -- 240

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '240' in i:
    lst.append(i)

print(lst)

['240']


In [ ]:
settlementid996 = election[ (election.settlementid == '996') &  (election.partyids.str.contains('240')) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
20828,1990,1,1990-03-25,"1990, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,3,7299,29,Szabad Demokraták Szövetsége,240,86
148256,1994,4,1994-05-08,"1994, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,71178,29,Szabad Demokraták Szövetsége,240,36
282932,1998,7,1998-05-10,"1998, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,8576,29,Szabad Demokraták Szövetsége,240,8
398756,2002,12,2002-04-16,"2002, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9225,34,SZDSZ,240,11
485432,2006,21,2006-04-09,"2006, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9847,80,"Szabad Demokraták Szövetsége, a Magyar Liberál...",240,8


### MDF -- 269

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '269' in i:
    lst.append(i)

print(lst)

['269', '269 | 16', '16 | 269']


In [ ]:
settlementid996 = election[ (election.settlementid == '996') &  (election.partyids.str.contains('269')) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
20833,1990,1,1990-03-25,"1990, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,3,7299,47,Magyar Demokrata Fórum,269,47
148266,1994,4,1994-05-08,"1994, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,71178,64,MDF,269,65
282933,1998,7,1998-05-10,"1998, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,8576,47,Magyar Demokrata Fórum,269,8
398760,2002,12,2002-04-16,"2002, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9225,85,MDF | FIDESZ,269 | 16,110
485429,2006,21,2006-04-09,"2006, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9847,47,Magyar Demokrata Fórum,269,10
560792,2010,24,2010-04-11,"2010, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,67756,47,Magyar Demokrata Fórum,269,2


### MSZP -- 53

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '53' in i:
    lst.append(i)

print(lst)

['53', '153', '253', '6 | 69 | 8 | 45 | 53', '45 | 8 | 6 | 69 | 53', '8 | 69 | 53 | 45 | 6', '53 | 45 | 6 | 8 | 69', '69 | 6 | 53 | 8 | 45', '45 | 69 | 53 | 6 | 8', '69 | 8 | 53 | 45 | 6', '53 | 45 | 69 | 8 | 6', '6 | 53 | 69 | 45 | 8', '6 | 45 | 69 | 53 | 8', '8 | 45 | 6 | 69 | 53', '6 | 69 | 53 | 45 | 8', '6 | 8 | 45 | 53 | 69', '53 | 69 | 6 | 45 | 8', '8 | 69 | 53 | 6 | 45', '53 | 69 | 45 | 8 | 6', '69 | 53 | 45 | 8 | 6', '8 | 53 | 45 | 69 | 6', '53 | 69 | 8 | 6 | 45', '53 | 45 | 8 | 6 | 69', '6 | 53 | 45 | 8 | 69', '53 | 6 | 69 | 45 | 8', '8 | 45 | 69 | 53 | 6', '8 | 6 | 69 | 53 | 45', '45 | 53 | 69 | 8 | 6', '8 | 53 | 69 | 6 | 45', '45 | 53 | 6 | 8 | 69', '45 | 8 | 69 | 53 | 6', '69 | 45 | 8 | 53 | 6', '69 | 6 | 45 | 53 | 8', '53 | 45 | 8 | 69 | 6', '6 | 45 | 8 | 53 | 69', '53 | 6 | 8 | 45 | 69', '69 | 6 | 8 | 45 | 53', '8 | 53 | 69 | 45 | 6', '53 | 69 | 8 | 45 | 6', '6 | 8 | 69 | 45 | 53', '8 | 45 | 53 | 69 | 6', '8 | 6 | 53 | 69 | 45', '53 | 8 | 45 | 69 | 6', '8 | 6 | 45 | 53 | 

In [ ]:
settlementid996 = election[ (election.year == '2014') & (election.settlementid == '2417') & 
                           ( (election.partyids.str.contains('^53$')) | 
                            (election.partyids.str.contains('^53 \|')) | 
                            (election.partyids.str.contains('\| 53$')) | 
                            (election.partyids.str.contains('\| 53 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
615658,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,6,63388,84,Együtt | PM | MSZP | MLP | DK,8 | 69 | 53 | 45 | 6,92
615676,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,7,63389,84,MSZP | MLP | DK | Együtt | PM,53 | 45 | 6 | 8 | 69,117
615694,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,8,63390,84,PM | DK | MSZP | Együtt | MLP,69 | 6 | 53 | 8 | 45,129
615712,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,9,63391,84,MLP | PM | MSZP | DK | Együtt,45 | 69 | 53 | 6 | 8,141
615730,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,10,63392,84,PM | Együtt | MSZP | MLP | DK,69 | 8 | 53 | 45 | 6,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618106,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,92,63460,84,MSZP | MLP | Együtt | DK | PM,53 | 45 | 8 | 6 | 69,119
618124,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,93,63461,84,Együtt | DK | PM | MSZP | MLP,8 | 6 | 69 | 53 | 45,92
618142,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,94,63462,84,Együtt | MLP | PM | MSZP | DK,8 | 45 | 69 | 53 | 6,141
618160,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,95,63463,84,PM | DK | MSZP | Együtt | MLP,69 | 6 | 53 | 8 | 45,77


### FKGP -- 18

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '18' in i:
    lst.append(i)

print(lst)

['182', '18', '186', '60718']


In [ ]:
settlementid996 = election[ (election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^18$')) | 
                            (election.partyids.str.contains('^18 \|')) | 
                            (election.partyids.str.contains('\| 18$')) | 
                            (election.partyids.str.contains('\| 18 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
20838,1990,1,1990-03-25,"1990, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,3,7299,77,"Független Kisgazda-, Földmunkás- és Polgári Párt",18,46
148261,1994,4,1994-05-08,"1994, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,71178,49,Független Kisgazdapárt,18,38
282934,1998,7,1998-05-10,"1998, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,8576,49,Független Kisgazdapárt,18,98
398755,2002,12,2002-04-16,"2002, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9225,18,FKGP,18,8
655843,2014,27,2014-04-06,"2014, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,77325,49,Független Kisgazdapárt,18,0


### Jobbik -- 24

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '24' in i:
    lst.append(i)

print(lst)

['240', '124', '144 | 243 | 230', '243 | 230 | 144', '230 | 243 | 144', '243 | 144 | 230', '144 | 230 | 243', '230 | 144 | 243', '24 | 43', '43 | 24', '24', '60724']


In [ ]:
settlementid996 = election[ (election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^24$')) | 
                            (election.partyids.str.contains('^24 \|')) | 
                            (election.partyids.str.contains('\| 24$')) | 
                            (election.partyids.str.contains('\| 24 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
485431,2006,21,2006-04-09,"2006, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9847,78,Jobbik | MIÉP,24 | 43,0
560790,2010,24,2010-04-11,"2010, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,67756,19,Jobbik,24,117
655838,2014,27,2014-04-06,"2014, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,77325,19,Jobbik,24,93
855985,2018,48,2018-04-08,"2018, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,69185,19,Jobbik,24,89


### LMP -- 30

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '30' in i:
    lst.append(i)

print(lst)

['144 | 243 | 230', '243 | 230 | 144', '230 | 243 | 144', '243 | 144 | 230', '144 | 230 | 243', '230 | 144 | 243', '30']


In [ ]:
settlementid996 = election[ (election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^30$')) | 
                            (election.partyids.str.contains('^30 \|')) | 
                            (election.partyids.str.contains('\| 30$')) | 
                            (election.partyids.str.contains('\| 30 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
560791,2010,24,2010-04-11,"2010, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,67756,43,Lehet Más a Politika,30,7
655847,2014,27,2014-04-06,"2014, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,77325,63,LMP,30,4
855986,2018,48,2018-04-08,"2018, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,69185,43,Lehet Más a Politika,30,3


### DK -- 6

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '6' in i:
    lst.append(i)

print(lst)

['26', '269', '16', '186', '166', '206', '146', '263', '266', '269 | 16', '16 | 269', '36', '16 | 26', '26 | 16', '66', '86', '6 | 69 | 8 | 45 | 53', '45 | 8 | 6 | 69 | 53', '8 | 69 | 53 | 45 | 6', '53 | 45 | 6 | 8 | 69', '69 | 6 | 53 | 8 | 45', '45 | 69 | 53 | 6 | 8', '69 | 8 | 53 | 45 | 6', '53 | 45 | 69 | 8 | 6', '6 | 53 | 69 | 45 | 8', '6 | 45 | 69 | 53 | 8', '8 | 45 | 6 | 69 | 53', '6 | 69 | 53 | 45 | 8', '6 | 8 | 45 | 53 | 69', '53 | 69 | 6 | 45 | 8', '8 | 69 | 53 | 6 | 45', '53 | 69 | 45 | 8 | 6', '69 | 53 | 45 | 8 | 6', '8 | 53 | 45 | 69 | 6', '53 | 69 | 8 | 6 | 45', '53 | 45 | 8 | 6 | 69', '6 | 53 | 45 | 8 | 69', '53 | 6 | 69 | 45 | 8', '8 | 45 | 69 | 53 | 6', '8 | 6 | 69 | 53 | 45', '45 | 53 | 69 | 8 | 6', '8 | 53 | 69 | 6 | 45', '45 | 53 | 6 | 8 | 69', '45 | 8 | 69 | 53 | 6', '69 | 45 | 8 | 53 | 6', '69 | 6 | 45 | 53 | 8', '53 | 45 | 8 | 69 | 6', '6 | 45 | 8 | 53 | 69', '53 | 6 | 8 | 45 | 69', '69 | 6 | 8 | 45 | 53', '8 | 53 | 69 | 45 | 6', '53 | 69 | 8 | 45 | 6', '6 | 8 | 6

In [ ]:
settlementid996 = election[ (election.settlementid == '2417') & 
                           ( (election.partyids.str.contains('^6$')) | 
                            (election.partyids.str.contains('^6 \|')) | 
                            (election.partyids.str.contains('\| 6$')) | 
                            (election.partyids.str.contains('\| 6 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
615658,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,6,63388,84,Együtt | PM | MSZP | MLP | DK,8 | 69 | 53 | 45 | 6,92
615676,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,7,63389,84,MSZP | MLP | DK | Együtt | PM,53 | 45 | 6 | 8 | 69,117
615694,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,8,63390,84,PM | DK | MSZP | Együtt | MLP,69 | 6 | 53 | 8 | 45,129
615712,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,9,63391,84,MLP | PM | MSZP | DK | Együtt,45 | 69 | 53 | 6 | 8,141
615730,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,10,63392,84,PM | Együtt | MSZP | MLP | DK,69 | 8 | 53 | 45 | 6,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805804,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,92,82499,199,Demokratikus Koalíció,6,17
805828,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,93,82500,199,Demokratikus Koalíció,6,18
805851,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,94,82501,199,Demokratikus Koalíció,6,26
805875,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,95,82502,199,Demokratikus Koalíció,6,12


### Momentum -- 60760

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '60760' in i:
    lst.append(i)

print(lst)

['60760']


In [ ]:
settlementid996 = election[ (election.settlementid == '2417') & 
                           ( (election.partyids.str.contains('^60760$')) | 
                            (election.partyids.str.contains('^60760 \|')) | 
                            (election.partyids.str.contains('\| 60760$')) | 
                            (election.partyids.str.contains('\| 60760 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
802608,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,6,78549,200,Momentum Mozgalom,60760,23
802631,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,7,78550,200,Momentum Mozgalom,60760,23
802655,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,8,78551,200,Momentum Mozgalom,60760,18
802679,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,9,78552,200,Momentum Mozgalom,60760,22
802702,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,10,78553,200,Momentum Mozgalom,60760,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805805,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,92,82499,200,Momentum Mozgalom,60760,26
805829,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,93,82500,200,Momentum Mozgalom,60760,27
805852,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,94,82501,200,Momentum Mozgalom,60760,24
805876,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 2. választókerület,31723,Kecskemét,2417,95,82502,200,Momentum Mozgalom,60760,11


### MSZMP -- 182

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '182' in i:
    lst.append(i)

print(lst)

['182']


In [ ]:
settlementid996 = election[ #(election.settlementid == '2417') & 
                           ( (election.partyids.str.contains('^182$')) | 
                            (election.partyids.str.contains('^182 \|')) | 
                            (election.partyids.str.contains('\| 182$')) | 
                            (election.partyids.str.contains('\| 182 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14
16,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,13661,35,Magyar Szocialista Munkáspárt,182,10
28,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,13662,35,Magyar Szocialista Munkáspárt,182,24
40,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,13663,35,Magyar Szocialista Munkáspárt,182,5
52,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,13664,35,Magyar Szocialista Munkáspárt,182,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122987,1990,1,1990-03-25,"1990, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,8,21167,35,Magyar Szocialista Munkáspárt,182,10
122999,1990,1,1990-03-25,"1990, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,9,21168,35,Magyar Szocialista Munkáspárt,182,2
123011,1990,1,1990-03-25,"1990, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,1,13058,35,Magyar Szocialista Munkáspárt,182,64
123023,1990,1,1990-03-25,"1990, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,2,13057,35,Magyar Szocialista Munkáspárt,182,2


### MSZDP -- 52

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '52' in i:
    lst.append(i)

print(lst)

['52', '152']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^52$')) | 
                            (election.partyids.str.contains('^52 \|')) | 
                            (election.partyids.str.contains('\| 52$')) | 
                            (election.partyids.str.contains('\| 52 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15
15,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,13661,31,Magyarországi Szociáldemokrata Párt,52,15
27,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,13662,31,Magyarországi Szociáldemokrata Párt,52,19
39,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,13663,31,Magyarországi Szociáldemokrata Párt,52,17
51,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,13664,31,Magyarországi Szociáldemokrata Párt,52,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603013,2010,24,2010-04-11,"2010, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentlászló,514,2,59635,17,MSZDP,52,0
603019,2010,24,2010-04-11,"2010, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentlőrinc,1108,1,68259,17,MSZDP,52,0
603025,2010,24,2010-04-11,"2010, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,1,59636,17,MSZDP,52,3
603031,2010,24,2010-04-11,"2010, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,1,13048,17,MSZDP,52,2


### ASZ -- 97

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '97' in i:
    lst.append(i)

print(lst)

['97']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^97$')) | 
                            (election.partyids.str.contains('^97 \|')) | 
                            (election.partyids.str.contains('\| 97$')) | 
                            (election.partyids.str.contains('\| 97 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
8,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,52,Agrárszövetség,97,20
20,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,13661,52,Agrárszövetség,97,23
32,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,13662,52,Agrárszövetség,97,14
44,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,13663,52,Agrárszövetség,97,8
56,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,13664,52,Agrárszövetség,97,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261498,1994,4,1994-05-08,"1994, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentlőrinc,1108,1,72026,52,Agrárszövetség,97,3
261509,1994,4,1994-05-08,"1994, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,1,72038,52,Agrárszövetség,97,10
261520,1994,4,1994-05-08,"1994, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,1,13056,52,Agrárszövetség,97,18
261531,1994,4,1994-05-08,"1994, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,2,13055,52,Agrárszövetség,97,0


### HK -- 133

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '133' in i:
    lst.append(i)

print(lst)

['133']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^133$')) | 
                            (election.partyids.str.contains('^133 \|')) | 
                            (election.partyids.str.contains('\| 133$')) | 
                            (election.partyids.str.contains('\| 133 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0
12,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,13661,3,Hazafias Választási Koalíció,133,0
24,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,13662,3,Hazafias Választási Koalíció,133,1
36,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,13663,3,Hazafias Választási Koalíció,133,2
48,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,13664,3,Hazafias Választási Koalíció,133,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122983,1990,1,1990-03-25,"1990, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,8,21167,3,Hazafias Választási Koalíció,133,105
122995,1990,1,1990-03-25,"1990, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,9,21168,3,Hazafias Választási Koalíció,133,53
123007,1990,1,1990-03-25,"1990, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,1,13058,3,Hazafias Választási Koalíció,133,37
123019,1990,1,1990-03-25,"1990, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,2,13057,3,Hazafias Választási Koalíció,133,10


### VP -- 258

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '258' in i:
    lst.append(i)

print(lst)

['258']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^258$')) | 
                            (election.partyids.str.contains('^258 \|')) | 
                            (election.partyids.str.contains('\| 258$')) | 
                            (election.partyids.str.contains('\| 258 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
5,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,36,Vállalkozók Pártja,258,9
17,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,13661,36,Vállalkozók Pártja,258,9
29,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,13662,36,Vállalkozók Pártja,258,5
41,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,13663,36,Vállalkozók Pártja,258,9
53,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,13664,36,Vállalkozók Pártja,258,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364599,1998,7,1998-05-10,"1998, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentlőrinc,1108,1,9010,36,Vállalkozók Pártja,258,0
364610,1998,7,1998-05-10,"1998, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,1,73287,36,Vállalkozók Pártja,258,0
364621,1998,7,1998-05-10,"1998, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,1,13054,36,Vállalkozók Pártja,258,1
364632,1998,7,1998-05-10,"1998, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,2,13053,36,Vállalkozók Pártja,258,0


### Munkaspart -- 199

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '199' in i:
    lst.append(i)

print(lst)

['199']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^199$')) | 
                            (election.partyids.str.contains('^199 \|')) | 
                            (election.partyids.str.contains('\| 199$')) | 
                            (election.partyids.str.contains('\| 199 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
123052,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,70829,55,Munkáspárt,199,24
123067,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,70830,55,Munkáspárt,199,13
123082,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,21338,55,Munkáspárt,199,10
123097,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,21339,55,Munkáspárt,199,12
123112,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,21340,55,Munkáspárt,199,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034757,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalasárszeg,97,1,86023,227,Magyar Munkáspárt,199,0
1034780,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentbalázs,1628,1,11834,227,Magyar Munkáspárt,199,0
1034803,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentjakab,1581,1,86026,227,Magyar Munkáspárt,199,0
1034826,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaújlak,723,1,86031,227,Magyar Munkáspárt,199,0


### KP -- 153

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '153' in i:
    lst.append(i)

print(lst)

['153']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^153$')) | 
                            (election.partyids.str.contains('^153 \|')) | 
                            (election.partyids.str.contains('\| 153$')) | 
                            (election.partyids.str.contains('\| 153 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
123048,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,70829,39,Köztársaság Párt,153,24
123063,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,70830,39,Köztársaság Párt,153,14
123078,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,21338,39,Köztársaság Párt,153,19
123093,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,21339,39,Köztársaság Párt,153,20
123108,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,21340,39,Köztársaság Párt,153,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256444,1994,4,1994-05-08,"1994, listás",Veszprém megye,3525,Veszprém megye 7. választókerület,26464,Veszprém,957,50,28699,39,Köztársaság Párt,153,25
256458,1994,4,1994-05-08,"1994, listás",Veszprém megye,3525,Veszprém megye 7. választókerület,26464,Veszprém,957,51,28700,39,Köztársaság Párt,153,6
256472,1994,4,1994-05-08,"1994, listás",Veszprém megye,3525,Veszprém megye 7. választókerület,26464,Veszprém,957,52,28701,39,Köztársaság Párt,153,9
256486,1994,4,1994-05-08,"1994, listás",Veszprém megye,3525,Veszprém megye 7. választókerület,26464,Veszprém,957,53,28702,39,Köztársaság Párt,153,0


### MIEP -- 43

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '43' in i:
    lst.append(i)

print(lst)

['144 | 243 | 230', '243 | 230 | 144', '230 | 243 | 144', '243 | 144 | 230', '144 | 230 | 243', '230 | 144 | 243', '43', '24 | 43', '43 | 24']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^43$')) | 
                            (election.partyids.str.contains('^43 \|')) | 
                            (election.partyids.str.contains('\| 43$')) | 
                            (election.partyids.str.contains('\| 43 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
123056,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,70829,65,Magyar Igazság és Élet Pártja,43,1
123071,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,70830,65,Magyar Igazság és Élet Pártja,43,4
123086,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,21338,65,Magyar Igazság és Élet Pártja,43,2
123101,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,21339,65,Magyar Igazság és Élet Pártja,43,5
123116,1994,4,1994-05-08,"1994, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,21340,65,Magyar Igazság és Élet Pártja,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034738,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalasárszeg,97,1,86023,65,Magyar Igazság és Élet Pártja,43,0
1034761,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentbalázs,1628,1,11834,65,Magyar Igazság és Élet Pártja,43,0
1034784,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentjakab,1581,1,86026,65,Magyar Igazság és Élet Pártja,43,0
1034807,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaújlak,723,1,86031,65,Magyar Igazság és Élet Pártja,43,0


### MDNP -- 192

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '192' in i:
    lst.append(i)

print(lst)

['192']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^192$')) | 
                            (election.partyids.str.contains('^192 \|')) | 
                            (election.partyids.str.contains('\| 192$')) | 
                            (election.partyids.str.contains('\| 192 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
261549,1998,7,1998-05-10,"1998, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,28973,28,Magyar Demokrata Néppárt,192,0
261561,1998,7,1998-05-10,"1998, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,28974,28,Magyar Demokrata Néppárt,192,1
261573,1998,7,1998-05-10,"1998, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,28975,28,Magyar Demokrata Néppárt,192,2
261585,1998,7,1998-05-10,"1998, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,28976,28,Magyar Demokrata Néppárt,192,2
261597,1998,7,1998-05-10,"1998, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,28977,28,Magyar Demokrata Néppárt,192,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364597,1998,7,1998-05-10,"1998, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentlőrinc,1108,1,9010,28,Magyar Demokrata Néppárt,192,11
364608,1998,7,1998-05-10,"1998, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,1,73287,28,Magyar Demokrata Néppárt,192,1
364619,1998,7,1998-05-10,"1998, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,1,13054,28,Magyar Demokrata Néppárt,192,3
364630,1998,7,1998-05-10,"1998, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,2,13053,28,Magyar Demokrata Néppárt,192,0


### Centrum -- 100 (2002-ben) 

### Centrum -- 5 (2006-ban)

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '100' in i:
    lst.append(i)

print(lst)

['100']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^100$')) | 
                            (election.partyids.str.contains('^100 \|')) | 
                            (election.partyids.str.contains('\| 100$')) | 
                            (election.partyids.str.contains('\| 100 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
373338,2002,12,2002-04-16,"2002, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,36595,9,Centrum,100,19
373345,2002,12,2002-04-16,"2002, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,36596,9,Centrum,100,17
373352,2002,12,2002-04-16,"2002, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,36598,9,Centrum,100,31
373359,2002,12,2002-04-16,"2002, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,36599,9,Centrum,100,17
373366,2002,12,2002-04-16,"2002, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,36600,9,Centrum,100,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456845,2002,12,2002-04-16,"2002, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentlőrinc,1108,1,74518,9,Centrum,100,6
456852,2002,12,2002-04-16,"2002, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalaszentmihály,1616,1,74535,9,Centrum,100,23
456859,2002,12,2002-04-16,"2002, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,1,13052,9,Centrum,100,9
456866,2002,12,2002-04-16,"2002, listás",Zala megye,3526,Zala megye 5. választókerület,26469,Zalatárnok,2711,2,13051,9,Centrum,100,0


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^5$')) | 
                            (election.partyids.str.contains('^5 \|')) | 
                            (election.partyids.str.contains('\| 5$')) | 
                            (election.partyids.str.contains('\| 5 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
456879,2006,21,2006-04-09,"2006, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,44264,27,Centrum Összefogás Magyarországért,5,2
456887,2006,21,2006-04-09,"2006, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,44265,27,Centrum Összefogás Magyarországért,5,4
456895,2006,21,2006-04-09,"2006, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,44266,27,Centrum Összefogás Magyarországért,5,1
456903,2006,21,2006-04-09,"2006, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,44267,27,Centrum Összefogás Magyarországért,5,7
456911,2006,21,2006-04-09,"2006, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,44268,27,Centrum Összefogás Magyarországért,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528131,2006,21,2006-04-09,"2006, listás",Szabolcs-Szatmár-Bereg megye,3522,Szabolcs-Szatmár-Bereg megye 9. választókerület,26446,Szamosszeg,1084,2,75524,27,Centrum Összefogás Magyarországért,5,1
528141,2006,21,2006-04-09,"2006, listás",Szabolcs-Szatmár-Bereg megye,3522,Szabolcs-Szatmár-Bereg megye 9. választókerület,26446,Tyukod,2878,1,75652,27,Centrum Összefogás Magyarországért,5,1
528151,2006,21,2006-04-09,"2006, listás",Szabolcs-Szatmár-Bereg megye,3522,Szabolcs-Szatmár-Bereg megye 9. választókerület,26446,Tyukod,2878,2,75653,27,Centrum Összefogás Magyarországért,5,1
528161,2006,21,2006-04-09,"2006, listás",Szabolcs-Szatmár-Bereg megye,3522,Szabolcs-Szatmár-Bereg megye 9. választókerület,26446,Tyukod,2878,3,75654,27,Centrum Összefogás Magyarországért,5,1


### PM -- 69

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '69' in i:
    lst.append(i)

print(lst)

['269', '269 | 16', '16 | 269', '6 | 69 | 8 | 45 | 53', '45 | 8 | 6 | 69 | 53', '8 | 69 | 53 | 45 | 6', '53 | 45 | 6 | 8 | 69', '69 | 6 | 53 | 8 | 45', '45 | 69 | 53 | 6 | 8', '69 | 8 | 53 | 45 | 6', '53 | 45 | 69 | 8 | 6', '6 | 53 | 69 | 45 | 8', '6 | 45 | 69 | 53 | 8', '8 | 45 | 6 | 69 | 53', '6 | 69 | 53 | 45 | 8', '6 | 8 | 45 | 53 | 69', '53 | 69 | 6 | 45 | 8', '8 | 69 | 53 | 6 | 45', '53 | 69 | 45 | 8 | 6', '69 | 53 | 45 | 8 | 6', '8 | 53 | 45 | 69 | 6', '53 | 69 | 8 | 6 | 45', '53 | 45 | 8 | 6 | 69', '6 | 53 | 45 | 8 | 69', '53 | 6 | 69 | 45 | 8', '8 | 45 | 69 | 53 | 6', '8 | 6 | 69 | 53 | 45', '45 | 53 | 69 | 8 | 6', '8 | 53 | 69 | 6 | 45', '45 | 53 | 6 | 8 | 69', '45 | 8 | 69 | 53 | 6', '69 | 45 | 8 | 53 | 6', '69 | 6 | 45 | 53 | 8', '53 | 45 | 8 | 69 | 6', '6 | 45 | 8 | 53 | 69', '53 | 6 | 8 | 45 | 69', '69 | 6 | 8 | 45 | 53', '8 | 53 | 69 | 45 | 6', '53 | 69 | 8 | 45 | 6', '6 | 8 | 69 | 45 | 53', '8 | 45 | 53 | 69 | 6', '8 | 6 | 53 | 69 | 45', '53 | 8 | 45 | 69 | 6', '8 | 6 |

In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^69$')) | 
                            (election.partyids.str.contains('^69 \|')) | 
                            (election.partyids.str.contains('\| 69$')) | 
                            (election.partyids.str.contains('\| 69 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
603059,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Felsőlajos,3091,1,68484,84,DK | PM | Együtt | MLP | MSZP,6 | 69 | 8 | 45 | 53,43
603077,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Fülöpháza,2885,1,6564,84,MLP | Együtt | DK | PM | MSZP,45 | 8 | 6 | 69 | 53,60
603095,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,6,63388,84,Együtt | PM | MSZP | MLP | DK,8 | 69 | 53 | 45 | 6,92
603113,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,7,63389,84,MSZP | MLP | DK | Együtt | PM,53 | 45 | 6 | 8 | 69,117
603131,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,8,63390,84,PM | DK | MSZP | Együtt | MLP,69 | 6 | 53 | 8 | 45,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034747,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalasárszeg,97,1,86023,204,Párbeszéd Magyarországért Párt | Magyar Szocia...,69 | 53,2
1034770,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentbalázs,1628,1,11834,204,Párbeszéd Magyarországért Párt | Magyar Szocia...,69 | 53,44
1034793,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentjakab,1581,1,86026,204,Magyar Szocialista Párt | Párbeszéd Magyarorsz...,53 | 69,2
1034816,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaújlak,723,1,86031,204,Magyar Szocialista Párt | Párbeszéd Magyarorsz...,53 | 69,4


### Egyutt -- 8 (2014-ben) 

### Egutt -- 9 (2018-ban) 

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '8' in i:
    lst.append(i)

print(lst)

['182', '258', '18', '186', '208', '238', '228', '148', '86', '38', '89', '6 | 69 | 8 | 45 | 53', '80', '45 | 8 | 6 | 69 | 53', '8 | 69 | 53 | 45 | 6', '53 | 45 | 6 | 8 | 69', '69 | 6 | 53 | 8 | 45', '45 | 69 | 53 | 6 | 8', '69 | 8 | 53 | 45 | 6', '53 | 45 | 69 | 8 | 6', '6 | 53 | 69 | 45 | 8', '6 | 45 | 69 | 53 | 8', '8 | 45 | 6 | 69 | 53', '6 | 69 | 53 | 45 | 8', '6 | 8 | 45 | 53 | 69', '53 | 69 | 6 | 45 | 8', '8 | 69 | 53 | 6 | 45', '53 | 69 | 45 | 8 | 6', '69 | 53 | 45 | 8 | 6', '8 | 53 | 45 | 69 | 6', '53 | 69 | 8 | 6 | 45', '53 | 45 | 8 | 6 | 69', '6 | 53 | 45 | 8 | 69', '53 | 6 | 69 | 45 | 8', '8 | 45 | 69 | 53 | 6', '8 | 6 | 69 | 53 | 45', '45 | 53 | 69 | 8 | 6', '8 | 53 | 69 | 6 | 45', '45 | 53 | 6 | 8 | 69', '45 | 8 | 69 | 53 | 6', '69 | 45 | 8 | 53 | 6', '69 | 6 | 45 | 53 | 8', '53 | 45 | 8 | 69 | 6', '6 | 45 | 8 | 53 | 69', '53 | 6 | 8 | 45 | 69', '69 | 6 | 8 | 45 | 53', '8 | 53 | 69 | 45 | 6', '53 | 69 | 8 | 45 | 6', '6 | 8 | 69 | 45 | 53', '8 | 45 | 53 | 69 | 6', '8 | 6 |

In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^8$')) | 
                            (election.partyids.str.contains('^8 \|')) | 
                            (election.partyids.str.contains('\| 8$')) | 
                            (election.partyids.str.contains('\| 8 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
603059,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Felsőlajos,3091,1,68484,84,DK | PM | Együtt | MLP | MSZP,6 | 69 | 8 | 45 | 53,43
603077,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Fülöpháza,2885,1,6564,84,MLP | Együtt | DK | PM | MSZP,45 | 8 | 6 | 69 | 53,60
603095,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,6,63388,84,Együtt | PM | MSZP | MLP | DK,8 | 69 | 53 | 45 | 6,92
603113,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,7,63389,84,MSZP | MLP | DK | Együtt | PM,53 | 45 | 6 | 8 | 69,117
603131,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,8,63390,84,PM | DK | MSZP | Együtt | MLP,69 | 6 | 53 | 8 | 45,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789917,2014,27,2014-04-06,"2014, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalasárszeg,97,1,68962,84,MLP | MSZP | PM | Együtt | DK,45 | 53 | 69 | 8 | 6,13
789935,2014,27,2014-04-06,"2014, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentbalázs,1628,1,11835,84,MSZP | Együtt | DK | PM | MLP,53 | 8 | 6 | 69 | 45,105
789953,2014,27,2014-04-06,"2014, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentjakab,1581,1,68965,84,PM | DK | MSZP | MLP | Együtt,69 | 6 | 53 | 45 | 8,14
789971,2014,27,2014-04-06,"2014, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaújlak,723,1,68970,84,MLP | PM | MSZP | DK | Együtt,45 | 69 | 53 | 6 | 8,6


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^9$')) | 
                            (election.partyids.str.contains('^9 \|')) | 
                            (election.partyids.str.contains('\| 9$')) | 
                            (election.partyids.str.contains('\| 9 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
603057,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Felsőlajos,3091,1,68484,75,Együtt 2014,9,2
603075,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Fülöpháza,2885,1,6564,75,Együtt 2014,9,1
603093,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,6,63388,75,Együtt 2014,9,2
603111,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,7,63389,75,Együtt 2014,9,1
603129,2014,27,2014-04-06,"2014, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,8,63390,75,Együtt 2014,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034739,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalasárszeg,97,1,86023,75,Együtt 2014,9,0
1034762,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentbalázs,1628,1,11834,75,Együtt 2014,9,5
1034785,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentjakab,1581,1,86026,75,Együtt 2014,9,0
1034808,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaújlak,723,1,86031,75,Együtt 2014,9,0


### MKKP -- 60740

In [ ]:
# Searching for '240' string in partyids 
lst = list()
for i in election.partyids.unique(): 
  if '60740' in i:
    lst.append(i)

print(lst)

['60740']


In [ ]:
settlementid996 = election[ #(election.settlementid == '996') & 
                           ( (election.partyids.str.contains('^60740$')) | 
                            (election.partyids.str.contains('^60740 \|')) | 
                            (election.partyids.str.contains('\| 60740$')) | 
                            (election.partyids.str.contains('\| 60740 \|')) 
                           ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
790000,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Felsőlajos,3091,1,69145,201,Magyar Kétfarkú Kutya Párt,60740,5
790023,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Fülöpháza,2885,1,6563,201,Magyar Kétfarkú Kutya Párt,60740,2
790046,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,6,78549,201,Magyar Kétfarkú Kutya Párt,60740,9
790069,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,7,78550,201,Magyar Kétfarkú Kutya Párt,60740,18
790093,2018,48,2018-04-08,"2018, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 1. választókerület,31722,Kecskemét,2417,8,78551,201,Magyar Kétfarkú Kutya Párt,60740,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034744,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalasárszeg,97,1,86023,201,Magyar Kétfarkú Kutya Párt,60740,0
1034767,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentbalázs,1628,1,11834,201,Magyar Kétfarkú Kutya Párt,60740,3
1034790,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentjakab,1581,1,86026,201,Magyar Kétfarkú Kutya Párt,60740,1
1034813,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaújlak,723,1,86031,201,Magyar Kétfarkú Kutya Párt,60740,0


## Splitting up ElectionPopulation by parties NEW!!!!

Fidesz -- 16

KDNP -- 26

SZDSZ -- 240 

MDF --269

MSZP -- 53

FKGP -- 18

Jobbik -- 24

LMP -- 30

DK -- 6

Momentum -- 60760

MSZMP -- 182

MSZDP -- 52 

Agrarszovetseg (ASZ) -- 97

Hazafias Koalicio -- 133

Vallalkozok Partja -- 258

Munkaspart -- 199 

Koztarsasagi Part (KP) -- 153

MIEP -- 43 

MDNP -- 192

Centrum -- 100 (2002-ben) 5 (2006-ban)

PM -- 69 

Egyutt -- 8 (2014-ben) 9 (2018-ban) 

MKKP - 60740


In [ ]:
partyLst = ['16', '26', '240', 
            '269', '53', '18', 
            '24', '30', '6', 
            '60760', '182', '52', 
            '97', '133', '258', 
            '199', '153', '43',
            '192', '100', '69',
            '8', '60740', '9', '5']

partyTranslateDict = {'16': 'Fidesz', '26': 'KDNP', '240': 'SZDSZ', 
                      '269': 'MDF', '53': 'MSZP', '18': 'FKGP',
                      '24': 'Jobbik', '30': 'LMP', '6': 'DK',
                      '60760': 'Momentum', '182': 'MSZMP', '52': 'MSZDP',
                      '97': 'ASZ', '133': 'HK', '258': 'VP', 
                      '199': 'Munkaspart', '153': 'KP', '43': 'MIEP',
                      '192': 'MDNP', '100': 'Centrum', '69': 'PM',
                      '8': 'Egyutt', '60740': 'MKKP', '9': 'Egyutt', '5': 'Centrum'} 

partyDict = dict.fromkeys(partyLst)

In [ ]:
partyDict

{'100': None,
 '133': None,
 '153': None,
 '16': None,
 '18': None,
 '182': None,
 '192': None,
 '199': None,
 '24': None,
 '240': None,
 '258': None,
 '26': None,
 '269': None,
 '30': None,
 '43': None,
 '5': None,
 '52': None,
 '53': None,
 '6': None,
 '60740': None,
 '60760': None,
 '69': None,
 '8': None,
 '9': None,
 '97': None}

In [ ]:
partyTranslateDict

{'100': 'Centrum',
 '133': 'HK',
 '153': 'KP',
 '16': 'Fidesz',
 '18': 'FKGP',
 '182': 'MSZMP',
 '192': 'MDNP',
 '199': 'Munkaspart',
 '24': 'Jobbik',
 '240': 'SZDSZ',
 '258': 'VP',
 '26': 'KDNP',
 '269': 'MDF',
 '30': 'LMP',
 '43': 'MIEP',
 '5': 'Centrum',
 '52': 'MSZDP',
 '53': 'MSZP',
 '6': 'DK',
 '60740': 'MKKP',
 '60760': 'Momentum',
 '69': 'PM',
 '8': 'Egyutt',
 '9': 'Egyutt',
 '97': 'ASZ'}

In [ ]:
electionPopulation['vote'] = electionPopulation['vote'].astype(np.int64)
allVotes=electionPopulation[['year','settlementid', 'vote']].groupby(['year','settlementid']).sum()
allVotes = allVotes.reset_index()
allVotes = allVotes.rename(columns={"vote": "sumvote"})
allVotes

,year,settlementid,sumvote
0,1990,10,929
1,1990,100,47641
2,1990,1000,1490
3,1990,1001,119
4,1990,1002,1116
...,...,...,...
25223,2018,995,1802
25224,2018,996,416
25225,2018,997,38
25226,2018,998,452


In [ ]:
trunc_elecYear_pop = elecYear_pop[['year', 'geonameid', 'geoid', 'name', 'value', 'datatypeid', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag']].drop_duplicates()
trunc_electionPopulation = electionPopulation[['year', 'settlementid', 'value', 'county', 'votePop_elig', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag']].drop_duplicates()
election_pop_small = trunc_elecYear_pop.merge(trunc_electionPopulation, left_on=['year', 'geoid', 'value', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag'], right_on=['year', 'settlementid', 'value', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag'])
election_pop_small = election_pop_small.rename(columns={"name": "settlement"}) # Raname the name column to settlement
election_pop_small = election_pop_small.drop(columns=['geoid']) #Remove geoid because it is the same as settlementid
election_pop_small

,year,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,settlementid,county,votePop_elig
0,1990,9692,Aba,4103,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3282
1,1994,9692,Aba,4202,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3362
2,1998,9692,Aba,4332,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3466
3,2002,9692,Aba,4526,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3621
4,2006,9692,Aba,4654,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3723
...,...,...,...,...,...,...,...,...,...,...
25223,2002,9759,Ősi,2244,1650,47.1539446896795,18.190410278261,2160,Veszprém megye,1795
25224,2006,9759,Ősi,2190,1650,47.1539446896795,18.190410278261,2160,Veszprém megye,1752
25225,2010,9759,Ősi,2169,1650,47.1539446896795,18.190410278261,2160,Veszprém megye,1735
25226,2014,9759,Ősi,2089,1650,47.1539446896795,18.190410278261,2160,Veszprém megye,1671


In [ ]:
geoname = pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/geoname.csv', dtype=str)
geoname = trim_all_columns(geoname) #Using Function 2 
bpDistrictGeoname = geoname[geoname['name'].str.match('Budapest')].sort_values(by = ['name']) #the 0th one is just simply Budapest, and we just mine and sort all the name which contain Budapest districts
bpDistrictGeoname.reset_index(drop=True, inplace=True)
bpDistrictGeoname[0:1]

,geonameid,name,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,16601,Budapest,1134,47.4825696830677,19.1592600789783


In [ ]:
df_electionVotes_single = None

for party in partyLst: 
  partyDict[party] = electionPopulation[
                            (electionPopulation.partyids.str.contains('^'+ party +'$')) | 
                            (electionPopulation.partyids.str.contains('^'+ party +' \|')) | 
                            (electionPopulation.partyids.str.contains('\| '+ party +'$')) | 
                            (electionPopulation.partyids.str.contains('\| '+ party +' \|')) 
                            ]


  votes_party = partyDict[party][['year','settlementid','vote']].groupby(['year','settlementid']).sum()
  votes_party = votes_party.reset_index()
  partyDict[party]=pd.merge(votes_party, election_pop_small, on=['settlementid', 'year'])
  partyDict[party].rename(columns={"vote": "vote_party"}, inplace = True)

  partyDict[party]= pd.merge(partyDict[party], allVotes, on=['settlementid', 'year'])

  #Convert some numerical string values to num
  partyDict[party]['vote_party'] = partyDict[party]['vote_party'].astype(np.int64)
  partyDict[party]['value'] = partyDict[party]['value'].astype(np.int64)
  partyDict[party]['votePop_elig'] = partyDict[party]['votePop_elig'].astype(np.int64)
  partyDict[party]['sumvote'] = partyDict[party]['sumvote'].astype(np.int64)

  partyDict[party]['vote_rate_party'] = partyDict[party]['vote_party'] / partyDict[party]['sumvote']
  partyDict[party]['vote_NOTparty'] = partyDict[party]['sumvote'] - partyDict[party]['vote_party']
  partyDict[party]['vote_rate_NOTparty'] = 1 - partyDict[party]['vote_rate_party']
  partyDict[party]['vote_rate_NOTparty_per_party'] = partyDict[party]['vote_NOTparty'] / partyDict[party]['vote_party']

  partyDict[party]['partyids'] = party 
  partyDict[party]['partyname'] = partyTranslateDict[party] 
  partyDict[party]['isCommonListViewer'] = 'No'



  if len( partyDict[party][partyDict[party]['value'] < partyDict[party]['sumvote']] ) != 0: 
    idxToDel = partyDict[party][partyDict[party]['value'] < partyDict[party]['sumvote']].index
    partyDict[party] = partyDict[party].drop(idxToDel)
  
  partyDict[party].sort_values(by=['year', 'settlement'], inplace = True)
  partyDict[party].reset_index(drop = True, inplace = True)


  Budapest_distr = partyDict[party][partyDict[party]['settlement'].str.contains('Budapest')] 
  
  toSumlst = ['vote_NOTparty', 'value', 'votePop_elig', 'vote_party', 'sumvote']
  Budapest_only= None

  for colIdx in range(len(toSumlst)): 
    temp_df = Budapest_distr[['year', toSumlst[colIdx] ]].groupby(['year']).sum().reset_index()

    if colIdx == 0:
      Budapest_only = temp_df

    else: 
      Budapest_only = pd.merge(Budapest_only, temp_df, on='year')

  Budapest_only['partyids'] = party
  Budapest_only['partyname'] = partyTranslateDict[party]
  Budapest_only['isCommonListViewer'] = 'No'
  Budapest_only['settlement'] = 'Budapest'
  Budapest_only['settlementid'] = '1134'
  Budapest_only['geonameid'] = '16601'
  Budapest_only['county'] = 'Budapest'
  Budapest_only['datatypeid'] = '1650'
  Budapest_only['wgs84_e_szelesseg'] = '47.4825696830677'
  Budapest_only['wgs84_k_hosszusag'] = '19.1592600789783'
  Budapest_only['vote_rate_party'] = Budapest_only['vote_party'] / (Budapest_only['vote_party'] + Budapest_only['vote_NOTparty'])
  Budapest_only['vote_rate_NOTparty'] = 1 - Budapest_only['vote_rate_party'] 
  Budapest_only['vote_rate_NOTparty_per_party'] = Budapest_only['vote_NOTparty'] / Budapest_only['vote_party']

  partyDict[party] = partyDict[party][['year', 'partyids', 'partyname', 'county', 'settlement', 'settlementid', 'geonameid', 'datatypeid', 'value', 
                                       'votePop_elig', 'sumvote', 'vote_NOTparty', 'vote_party', 'vote_rate_NOTparty', 
                                       'vote_rate_party', 'vote_rate_NOTparty_per_party', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag', 'isCommonListViewer']]

  Budapest_only = Budapest_only[partyDict[party].columns]

  concated = pd.concat([partyDict[party], Budapest_only])
  concated = concated.replace('nan', np.nan) 
  df_electionVotes_single = pd.concat([df_electionVotes_single, concated])
  
  df_electionVotes_single['vote_rate_NOTparty'] = round(df_electionVotes_single['vote_rate_NOTparty'], 4)
  df_electionVotes_single['vote_rate_party'] = round(df_electionVotes_single['vote_rate_party'], 4)
  df_electionVotes_single['vote_rate_NOTparty_per_party'] = round(df_electionVotes_single['vote_rate_NOTparty_per_party'], 4)

df_electionVotes_single.sort_values(by=['partyname', 'year', 'settlement'], inplace = True)
df_electionVotes_single.reset_index(drop=True, inplace=True)




  






  



In [ ]:
df_electionVotes_single

,year,partyids,partyname,county,settlement,settlementid,geonameid,datatypeid,value,votePop_elig,sumvote,vote_NOTparty,vote_party,vote_rate_NOTparty,vote_rate_party,vote_rate_NOTparty_per_party,wgs84_e_szelesseg,wgs84_k_hosszusag,isCommonListViewer
0,1990,97,ASZ,Fejér megye,Aba,1512,9692,1650,4103,3282,1530,1432,98,0.9359,0.0641,14.6122,47.0343556977497,18.524754962635,No
1,1990,97,ASZ,Baranya megye,Abaliget,1035,16239,1650,645,516,266,259,7,0.9737,0.0263,37.0000,46.1438461930372,18.1181108415909,No
2,1990,97,ASZ,Borsod-Abaúj-Zemplén megye,Abaújalpár,1341,17191,1650,136,109,94,74,20,0.7872,0.2128,3.7000,48.3061466428872,21.234096519055,No
3,1990,97,ASZ,Borsod-Abaúj-Zemplén megye,Abaújkér,2420,10500,1650,709,567,380,354,26,0.9316,0.0684,13.6154,48.3050378371762,21.2002207816169,No
4,1990,97,ASZ,Borsod-Abaúj-Zemplén megye,Abaújlak,82,16998,1650,137,110,96,91,5,0.9479,0.0521,18.2000,48.404816687476,20.9570494709898,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238354,1998,258,VP,Bács-Kiskun megye,Újsolt,664,17352,1650,191,153,107,106,1,0.9907,0.0093,106.0000,46.8729980275511,19.1193296356278,No
238355,1998,258,VP,Bács-Kiskun megye,Újtelek,3092,16009,1650,540,432,320,320,0,1.0000,0.0000,inf,46.5944423187449,19.0567541642347,No
238356,1998,258,VP,Zala megye,Újudvar,2955,10551,1650,1007,806,404,404,0,1.0000,0.0000,inf,46.5404968230274,16.9903362156209,No
238357,1998,258,VP,Somogy megye,Újvárfalva,511,15950,1650,393,314,140,140,0,1.0000,0.0000,inf,46.4336694546273,17.5742979795082,No


In [ ]:
df_electionVotes_single.to_csv(path + 'electionListVotes_Single.csv', index = False)

## **CommonParty List**

In [ ]:
elsov = election[election.year == '2018']
elsov = elsov_partok = elsov[['partylist', 'partyids', 'partylistid']].drop_duplicates(subset=['partylistid'])
elsov[elsov.partyids.str.contains('\| ')]

,partylist,partyids,partylistid
789996,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,86
790003,Párbeszéd Magyarországért Párt | Magyar Szocia...,69 | 53,204
